In [1]:
from tools.pipeline.config import ONEDRIVE_PATH
import os, json
from tqdm import tqdm
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

## 분류 및 문제 작성 완료

In [2]:
with open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '9_multiple_to_essay', 'essay_w_keyword.json'), 'r', encoding='utf-8') as f:
    full_explanation = json.load(f)

In [ ]:
# system_prompt = f"""
# 당신의 임무는 제공된 자료를 근거로 서술형 모범답안을 작성하는 것입니다. 사실 판단과 개념은 반드시 ‘선지별 해설’과 ‘원래 정답’에서 도출하세요. ‘원래 질문/선지’는 정오판단의 맥락 이해에만 사용하고, 최종 답안에는 일절 언급하지 마세요.

# 입력:
# - 서술형 질문: {{서술형 질문}}
# - 키워드: {{키워드}}
# - 원래 질문: {{원래 질문}}
# - 원래 선지: {{원래 선지}}
# - 원래 정답: {{원래 정답}}
# - 선지별 해설: {{선지별 해설}}

# 작성 지침:
# 1) 정답 근거 수립: ‘선지별 해설’과 ‘원래 정답’에서 해당 주제의 핵심 명제를 추출해 정합적으로 통합하라. 상충 가능성이 있을 경우 ‘선지별 해설’을 사실 기준으로 삼아라.
# 2) 키워드 사용: 제공된 모든 키워드를 문장 속에 자연스럽게 최소 1회 이상 포함하라. 단어 형태는 그대로 사용하되 문맥에 맞게 활용하라.
# 3) 내용 범위: 다음을 반드시 포함하라.
#    - 공급자 역할(가격 설정자/수용자 여부 등)
#    - 장기균형 이윤(경제학적 이윤·정상이윤 여부)
#    - 생산량 수준(평균비용곡선의 최저점 대비 위치)
#    - 상품 차별화의 성격과 상품 대체성의 정도
# 4) 표현 방식: 학술적이되 명료한 한국어로, 단락형 서술문으로 작성하라. 원래 문항의 번호, 선택지, 수식·그래프 표기는 노출하지 말라. 불필요한 사례·비유는 지양하라.
# 5) 분량: 5~8문장, 약 250~400자 내외.
# 6) 검증: 문장 간 논리적 일관성을 확인하고, 상호 모순(예: 가격 수용자/설정자 혼동, 장기 이윤에 대한 상반 진술 등)이 없도록 마무리하라.

# 출력:
# - 서술형 모범답안 본문만 제출한다. 제목, 머리말, 불릿, 번호, 인용, 메타설명은 금지한다.
# """

In [3]:
system_prompt = f"""
당신은 주어진 여러 정보를 조합하여 서술형 문제에 대한 '모범답안'을 생성하는 AI입니다.

**[역할]**
- '서술형 질문'의 요구사항을 정확히 파악합니다.
- '원래 질문/선지/정답/해설'에서 서술형 질문에 답변하는 데 필요한 핵심 정보를 추출합니다.
- 추출한 정보와 '키워드'를 논리적으로 엮어 하나의 완성된 글로 재구성합니다.

**[수행 절차]**
1.  **주제 파악:** '서술형 질문'과 '키워드'를 통해 모범답안이 다루어야 할 핵심 주제와 포함해야 할 요소를 확인합니다.
2.  **정보 추출:** '원래 선지'와 '선지별 해설'을 분석하여 각 '키워드'에 해당하는 구체적인 내용을 찾아냅니다.
3.  **논리적 구성:** 추출한 정보들을 활용하여 '서술형 질문'에 대한 답변이 되도록 문장을 자연스럽게 연결하고 문단을 구성합니다. 이때, 모든 '키워드'가 포함되어야 합니다.
4.  **답안 생성:** 위의 과정을 거쳐 최종 '모범답안'을 작성합니다.

**[규칙]**
- **절대 외부 지식을 사용하지 마세요.** 반드시 '원래 질문/선지/정답/해설'에 명시된 정보만을 근거로 답안을 작성해야 합니다.
- 제시된 모든 '키워드'를 반드시 답안에 포함해야 합니다.
- 최종 결과물은 "모범답안:"으로 시작하는 완성된 형태의 글이어야 합니다.
"""

In [4]:
# 추상화키워드: 99/859 완료, best_ans_추상화.json
for q in tqdm(full_explanation):
    user_prompt = f"""
========= 문제 ========
- 서술형 질문: {q['essay_question']}
- 키워드: {q['essay_keyword']}
- 원래 질문: {q['question']}
- 원래 선지: {q['options']}
- 원래 정답: {q['answer']}
- 선지별 해설: {q['explanation']}
"""
    response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'google/gemini-3-pro-preview')
    q['essay_answer'] = response.replace('모범답안:', '').strip()
    # break

100%|██████████| 859/859 [4:47:55<00:00, 20.11s/it]


In [5]:
with open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '9_multiple_to_essay', 'best_ans.json'), 'w', encoding='utf-8') as f:
    json.dump(full_explanation, f, ensure_ascii=False, indent=4)

In [6]:
len(full_explanation)

859